<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/transformers/finetuning_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://huggingface.co/transformers/examples.html#roberta-bert-and-masked-language-modeling

In [0]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Tue Apr 28 15:46:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================

In [0]:
# !pip install transformers

In [0]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 24978 (delta 43), reused 23 (delta 4), pack-reused 24889
Receiving objects: 100% (24978/24978), 15.20 MiB | 9.20 MiB/s, done.
Resolving deltas: 100% (17522/17522), done.


In [0]:
import os
os.chdir("transformers")
!pip install .

In [0]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
!unzip wikitext-2-raw-v1.zip

--2020-04-28 16:20:00--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.13.102
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.13.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip’

wikitext-2-raw-v1.z 100%[===================>]   4.50M  3.06MB/s    in 1.5s    

2020-04-28 16:20:03 (3.06 MB/s) - ‘wikitext-2-raw-v1.zip’ saved [4721645/4721645]



In [0]:
!python examples/run_language_modeling.py \
    --output_dir=output \
    --model_type=roberta \
    --model_name_or_path=roberta-base \
    --do_train \
    --train_data_file=wikitext-2-raw/wiki.test.raw \
    --do_eval \
    --eval_data_file=wikitext-2-raw/wiki.test.raw \
    --mlm

2020-04-28 16:20:56.756318: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/28/2020 16:20:58 - INFO - transformers.training_args -   PyTorch: setting up devices
04/28/2020 16:20:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/28/2020 16:20:58 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='output', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1)
04/28/2020 16:20:

In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
ROBERTA_PATH = INPUT_PATH + 'tf-roberta/'
EMOTION_PATH = INPUT_PATH + 'emotion/'

In [0]:
import pandas as pd, re, numpy as np
def remove_html(text):
    text = re.sub("&quot;", '"', text)
    text = re.sub("&gt;", ">", text)
    text = re.sub("&lt;", "<", text)
    text = re.sub("&le;", "≤", text)
    text = re.sub("&ge;", "≥", text)
    text = re.sub("&amp;", "&", text)
    return text

emotion_df=pd.read_csv(EMOTION_PATH + 'text_emotion.csv')
emotion_df['content'] = emotion_df['content'].astype(str)
emotion_df["content"] = emotion_df["content"].apply(lambda x: " ".join([i for i in x.split(" ") if "@" not in i]))
emotion_df["content"] = emotion_df["content"].apply(remove_html)

In [0]:
from sklearn.model_selection import train_test_split
emotion_train, emotion_test = train_test_split(emotion_df, test_size=0.1, random_state=333)

In [0]:
def to_text(df, filename):
    f = open(filename, 'w')
    for c in df["content"].values:
        f.write(" " + c.strip() + "\n\n")
    f.close()
to_text(emotion_test, "../emotion.test.txt")
to_text(emotion_train, "../emotion.train.txt")


In [0]:
!python examples/run_language_modeling.py \
    --output_dir=output_twitter \
    --model_type=roberta \
    --model_name_or_path=roberta-base \
    --do_train \
    --train_data_file=../emotion.test.txt \
    --do_eval \
    --eval_data_file=../emotion.train.txt \
    --mlm

2020-04-28 16:53:57.435816: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/28/2020 16:53:59 - INFO - transformers.training_args -   PyTorch: setting up devices
04/28/2020 16:53:59 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/28/2020 16:53:59 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='output_twitter', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1)
04/28/202